# 5.3. Inpainting

## Evaluation

Inpainting was tested to determine the extent to which **images** and **texts** can be created on food packaging. 

For this purpose, **pure inpainting** and **inpainting in combination with dreambooth** were used, resulting in a total of **four evaluations** in each case.

For each training, 50 output images were generated, which were rated as “achieved” and “not achieved” in the following categories:

1. **brand coherence**
2. **target design**
3. **visual aesthetics**

---


**Find the evaluated image-grids under:**

[Anhang 04_Inpainting](anhang/anhang%2004_inpainting/)

[Anhang 06_Inpainting Nesquik](anhang/anhang%2006_nesquik_inpainting/)

**And the Inpainting masks under:**

[ControlNet Mask](assets/)

## Setup

In [ ]:
%env HF_HOME=/cluster/user/ehoemmen/.cache
%env HF_DATASETS_CACHE=/cluster/user/ehoemmen/.cache

In [ ]:
pip install diffusers invisible_watermark transformers accelerate safetensors matplotlib -q --upgrade

In [ ]:
#Image Grid
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid

## Inference

In [ ]:
import torch
from diffusers import StableDiffusionXLInpaintPipeline
from diffusers.utils import load_image
import matplotlib.pyplot as plt

pipe = StableDiffusionXLInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True,
    cache_dir="/cluster/user/ehoemmen/.cache"
)
pipe.enable_sequential_cpu_offload()

# load fine-tuned LoRa-weights
pipe.load_lora_weights("erikhsos/cbbier_06-15-images_LoRA_lr1-4_2000")

# unload LoRa-weights
#pipe.unload_lora_weights()

In [ ]:
from PIL import Image
import os

img_url = load_image(
# '/cluster/user/ehoemmen/development/tests_sonstiges/05_Masterarbeit/03_inpainting/campusbier_input.png'
)

mask_url = load_image(
'/cluster/user/ehoemmen/development/tests_sonstiges/05_Masterarbeit/03_inpainting/inpainting_mask_schmied.png'   #enter path
)
init_image = load_image(img_url).convert("RGB")
mask_image = load_image(mask_url).convert("RGB")

num_images = 6

# select manual seed, if necessary
#generator = torch.manual_seed(493)

prompt = ["sliced lemon drawing with a leaf, beer label design"] * num_images
neg_prompt = "drawing of a building"

image = pipe(
    prompt=prompt,
    negative_prompt=neg_prompt, 
    image=init_image, mask_image=mask_image,
    #generator=generator, 
    num_inference_steps=25, 
    strength=0.80,
    guidance_scale=5.0
).images

grid = image_grid(image, rows=1, cols=num_images)

grid

## Results

**Best** inpainting results were archieved when using **inpainting x dreambooth** for image generation. In combination with the fine-tuned LoRa-weights the results match the original packaging design best.
It's possible to generate images with **inpainting only** but the relevant designs decrease a lot.

It's **not** possible to generate **text** on food packagings.